# Extending the BERT architecture for hate speech classification
#### In this notebook we conduct our experiments to extend the BERT architecture. The individual extensions (CNN; GNN/GCN; FC; RNN; LSTM) are later divided into individual notebooks for better traceability.


## Imports

In [ ]:
%pip install torch transformers datasets tf_keras transformers[torch] accelerate>=0.26.0 seaborn imblearn matplotlib numpy pandas scikit-learn optuna joblib wordcloud bs4 torch_geometric tqdm torchviz

In [ ]:
%pip install torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-2.6.0+cu118.html # change torch and cuda version to match yours

In [ ]:
import optuna
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
from optuna.pruners import HyperbandPruner
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset
from datasets import Dataset as data
from sklearn.metrics import (
    accuracy_score
    , precision_recall_fscore_support
    , matthews_corrcoef
    , fbeta_score
    , confusion_matrix
)
from transformers import AutoTokenizer, BertModel, Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification
import logging
import seaborn as sns
from bs4 import BeautifulSoup
from torch_geometric.data import Data
from torch_geometric.data import Batch
from transformers import DataCollatorWithPadding
from collections import Counter
from torch.utils.data import DataLoader
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, global_mean_pool
import networkx as nx
import spacy #python -m spacy download de_core_news_sm
import torch.nn.functional as F
from tqdm import tqdm
import os
from safetensors.torch import load_file

## Define the type of BERT Model
### We will use "bert-base-german-cased" across all experiments

In [ ]:
MODEL_NAME = "bert-base-german-cased"

## Miscellaneous

In [ ]:
logging.basicConfig(
    level=logging.INFO,  
    format="%(asctime)s - %(levelname)s - %(message)s",
    force=True
)
logger = logging.getLogger(__name__)
torch.cuda.empty_cache()

In [ ]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
logger.info(f"Using device: {device}")
if torch.cuda.is_available():
    logger.info(f"GPU Name: {torch.cuda.get_device_name(0)}")

## Co-Occurence Graph builder with cleaning

In [ ]:
nlp = spacy.load("de_core_news_sm")
stop_words = set(stopwords.words("german"))

def clean_german_tokens(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stop_words]

def build_vocab(texts, min_freq=1):
    from collections import Counter
    counter = Counter()
    for text in texts:
        tokens = clean_german_tokens(text)
        counter.update(tokens)

    vocab_tokens = [token for token, freq in counter.items() if freq >= min_freq]
    vocab = {token: idx for idx, token in enumerate(sorted(vocab_tokens))}
    return vocab

def build_graph_for_text(text, vocab):
    tokens = clean_german_tokens(text)
    node_ids = [vocab[token] for token in tokens if token in vocab]
    #print("Cleaned tokens:", tokens)

    if len(node_ids) == 0:
        #print(f"[Skipping OOV text]: '{text}'")
        return None

    x = torch.tensor(node_ids, dtype=torch.long).unsqueeze(1)

    edge_index = []
    for i in range(len(node_ids)):
        for j in range(i + 1, min(i + 3, len(node_ids))):  # sliding window
        #for j in range(i + 1, len(node_ids)): # complete
            edge_index.append([i, j])
            edge_index.append([j, i])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    assert max(node_ids) < len(vocab), f"Token index {max(node_ids)} exceeds vocab size {len(vocab)}"


    return Data(x=x, edge_index=edge_index)

## Cleaning of the texts
Necessary for precise training

In [ ]:
def normalize_insults(text):
    def fix_word(word):
        if re.fullmatch(r'(?:[a-zA-Z][\W_]{0,2}){2,}[a-zA-Z]', word):
            return re.sub(r'[\W_]+', '', word)
        else:
            return word
    words = text.split()
    words = [fix_word(word) for word in words]
    return ' '.join(words)

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = normalize_insults(text)
    return text

## Loading and Splitting of the Dataset
For our research we do not oversample the training set. We will be using the calculated class weights across all models.

In [ ]:
csv_path = "path/to/your/hatespeech_dataset" 
df = pd.read_csv(csv_path)
df = df[["text", "label_hs", "split_all"]]
df["text"] = df["text"].astype(str)
df["text"] = df["text"].apply(clean_text)
print("after cleaning", df.head())

train_df = df[df["split_all"] == "train"][["text", "label_hs"]].reset_index(drop=True)
test_df = df[df["split_all"] == "test"][["text", "label_hs"]].reset_index(drop=True)
val_df = df[df["split_all"] == "val"][["text", "label_hs"]].reset_index(drop=True)

average_word_count = train_df["text"].apply(lambda x: len(x.split())).mean()
print(f"Average number of words per text: {average_word_count:.2f}")

word_counts = train_df["text"].apply(lambda x: len(x.split()))
print(f"Max words: {word_counts.max()}, Min words: {word_counts.min()}")
print(word_counts.describe())  

plt.hist(word_counts, bins=50)
plt.title("Text Length Distribution (in words)")
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.show()

print("Verteilung der Labels:")
print(train_df["label_hs"].value_counts(normalize=True))
print(f"Train Data: {len(train_df)}")
print(f"Validation Data: {len(val_df)}")
print(f"Test Data: {len(test_df)}")

'''
Graph Builder for GNN
'''
vocab = build_vocab(train_df["text"].tolist(), min_freq=2)
print(f"Vocabulary size: {len(vocab)}")

full_df = pd.concat([train_df, test_df, val_df], ignore_index=True)
print(full_df.head())

# Class Weights
print("Verteilung:")
print(train_df['label_hs'].value_counts())
print(train_df["label_hs"].value_counts(normalize=True))
class_counts = train_df['label_hs'].value_counts().sort_index()
total_samples = class_counts.sum()
num_classes = len(class_counts)
boost = 1.5
class_weights = total_samples / (num_classes * class_counts)
class_weights[1] *= boost
class_weights = torch.tensor(class_weights.values, dtype=torch.float32).to(device)
logger.info(f"Computed class weights: {class_weights}")
print(f"Class weights: {class_weights}")

print("Anzahl Hate Speech und No Hate Speech:")
print(train_df['label_hs'].value_counts())
sns.countplot(data=train_df, x="label_hs")
plt.title("Verteilung nach Oversampling")
plt.show()

## Tokenization of the split data records and conversion into a BERT-compatible data record

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_texts(texts, tokenizer, max_length=128):  
    return tokenizer(
        list(texts), 
        truncation=True, 
        padding='max_length',  
        max_length=max_length, 
        return_tensors="pt", 
        return_attention_mask=True  
    )

def tokenize_and_map(df, tokenizer, max_length=128):
    encodings = tokenize_texts(df['text'].tolist(), tokenizer, max_length)
    return encodings

train_df = train_df.rename(columns={"label_hs": "labels"})
val_df = val_df.rename(columns={"label_hs": "labels"})
test_df = test_df.rename(columns={"label_hs": "labels"})

train_encodings = tokenize_and_map(train_df, tokenizer)
val_encodings = tokenize_and_map(val_df, tokenizer)
test_encodings = tokenize_and_map(test_df, tokenizer)

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}  
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = HateSpeechDataset(train_encodings, train_df['labels'].values) 
val_dataset = HateSpeechDataset(val_encodings, val_df['labels'].values)  
test_dataset = HateSpeechDataset(test_encodings, test_df['labels'].values)  

print(train_dataset[130])

## Dataset creation for Graph Tasks: Tokenization of the split data records and conversion into a BERT-compatible data record

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_texts(texts, tokenizer, max_length=128):  
    return tokenizer(
        list(texts),
        truncation=True,  
        padding='max_length',
        max_length=max_length,  
        return_tensors="pt", 
        return_attention_mask=True  
    )

def tokenize_and_map(df, tokenizer, max_length=128):
    encodings = tokenize_texts(df['text'].tolist(), tokenizer, max_length)
    return encodings

train_df = train_df.rename(columns={"label_hs": "labels"})
val_df = val_df.rename(columns={"label_hs": "labels"})
test_df = test_df.rename(columns={"label_hs": "labels"})

train_encodings = tokenize_and_map(train_df, tokenizer)
val_encodings = tokenize_and_map(val_df, tokenizer)
test_encodings = tokenize_and_map(test_df, tokenizer)

class HateSpeechDualDataset(Dataset):
    def __init__(self, df, encodings, vocab):
        self.encodings = encodings
        self.texts = df["text"].tolist()
        self.labels = df["labels"].tolist()
        self.vocab = vocab

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)

        graph = build_graph_for_text(self.texts[idx], self.vocab)
        if graph is None:
            graph = Data(x=torch.tensor([[0]]), edge_index=torch.tensor([[0, 0], [0, 0]]))  # fallback
        item['gnn_graph'] = graph

        return item

print(len(vocab))
train_dataset = HateSpeechDualDataset(train_df, train_encodings, vocab)
val_dataset = HateSpeechDualDataset(val_df, val_encodings, vocab)
test_dataset = HateSpeechDualDataset(test_df, test_encodings, vocab)

print(train_dataset[130])


def collate_fn(batch):
    # Debugging
    #print("Batch Keys:", batch[0].keys())
    hf_features = [
        {
            "input_ids": item["input_ids"],
            "attention_mask": item["attention_mask"],
            "labels": item["labels"],
        }
        for item in batch
    ]

    try:
        gnn_graphs = [item["gnn_graph"] for item in batch]
    except KeyError as e:
        print(f"KeyError: {e}. Batch sample does not contain 'gnn_graph' key.")
        raise e

    graph_batch = Batch.from_data_list(gnn_graphs)

    hf_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    hf_batch = hf_data_collator(hf_features)

    hf_batch["gnn_graph"] = graph_batch

    return hf_batch


print(train_dataset[0])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)
print(train_dataset[14])

## Graph Vizualisation

In [ ]:
def visualize_graph(graph_data, vocab):
    G = nx.Graph()

    edges = graph_data.edge_index.t().tolist()
    G.add_edges_from(edges)

    node_features = graph_data.x.squeeze().tolist()
    if not isinstance(node_features, list):
        node_features = [node_features] 

    node_labels = {i: vocab.get(node_features[i], f"id:{node_features[i]}") for i in range(len(node_features))}

    pos = nx.spring_layout(G)

    nx.draw(G, pos, with_labels=True, labels=node_labels, node_color='lightblue',
            node_size=1200, font_size=10, font_weight='bold', edge_color='gray')
    plt.title("Co-occurrence Graph")
    plt.show()

graph = train_dataset[190]['gnn_graph']
visualize_graph(graph, vocab)

def visualize_graph_with_words(graph_data, vocab):
    idx_to_word = {v: k for k, v in vocab.items()}
    
    node_ids = graph_data.x.squeeze().tolist()
    if isinstance(node_ids, int):
        node_ids = [node_ids]

    node_labels = {i: idx_to_word[node_id] for i, node_id in enumerate(node_ids)}

    G = nx.Graph()
    edge_index = graph_data.edge_index.numpy()

    for src, dst in edge_index.T:
        G.add_edge(src, dst)

    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, labels=node_labels, node_color='lightblue', node_size=1200, font_size=10)
    plt.title("Co-occurrence Graph with Word Labels")
    plt.show()

visualize_graph_with_words(graph, vocab)

tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased') 
input_ids = train_dataset[190]['input_ids']
reconstructed_text = tokenizer.decode(input_ids, skip_special_tokens=True)
print(reconstructed_text)

inv_vocab = {idx: token for token, idx in vocab.items()}
graph = train_dataset[190]['gnn_graph']
token_ids = graph.x.squeeze().tolist()  
tokens = [inv_vocab.get(tid, "<UNK>") for tid in token_ids]
print("Tokens from graph:", tokens)

## WordCloud

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('german'))

all_text = " ".join(full_df["text"].astype(str).tolist())

wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    max_words=200,
    stopwords=stop_words,
).generate(all_text)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud of Dataset", fontsize=20)
plt.show()

## Graph

In [ ]:
vocab_size = len(vocab)
print(vocab_size)

class BertGNNClassifier(nn.Module):
    def __init__(self, bert_model_name, hidden_dim, num_labels, dropout_rate=0.5, class_weights=None):
        super(BertGNNClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        self.node_embedding = nn.Embedding(vocab_size, 128) # embeddings add semantic information, important for GCN, neighborhood aggregators, etc.
        #self.gnn1 = SAGEConv(128, hidden_dim, aggr="lstm")  
        #self.gnn2 = SAGEConv(hidden_dim, hidden_dim, aggr="lstm")
        self.gnn1 = GATConv(128, hidden_dim)  
        self.gnn2 = GATConv(hidden_dim, hidden_dim)
        print(hidden_dim)

        self.fc = nn.Linear(self.bert.config.hidden_size + hidden_dim, num_labels)

        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids, attention_mask, graphs, labels=None):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = bert_outputs.last_hidden_state[:, 0]  

        gnn_outputs = []

        for graph in graphs.to_data_list():
            x = graph.x.view(-1)  # Flatten node indices
            #graph = graph.sort(sort_by_row=False) # for SAGEConv
            #if graph.edge_index.numel() == 0:
            #    zero_tensor = torch.zeros((1, self.gnn2.out_channels), device=cls_output.device) # for SAGEConv with LSTM: When Graph does not exist for a certain instance inside of the batch
            #    gnn_outputs.append(zero_tensor)
            #    continue
            assert graph.x.max().item() < vocab_size, "Node ID exceeds vocab size!"

            if x.max().item() >= vocab_size:
                print(x)
                print(f"Node ID out of bounds: max={x.max().item()} vs vocab size {vocab_size}")
                raise ValueError(f"Graph node ID out of range: {x.min().item()} - {x.max().item()} vs vocab size {vocab_size}")
            if x.min().item() < 0:
                print("Invalid negative node ID!")
                raise ValueError("Node ID out of bounds: Negative value encountered!")

            x = self.node_embedding(x) 
            x = F.relu(self.gnn1(x, graph.edge_index))
            x = F.relu(self.gnn2(x, graph.edge_index))
            pooled = global_mean_pool(x, batch=torch.zeros(x.size(0), dtype=torch.long, device=x.device))

            gnn_outputs.append(pooled)

        gnn_outputs = torch.stack(gnn_outputs).squeeze(1)  

        combined = torch.cat((cls_output, gnn_outputs), dim=1)
        combined = self.dropout(combined)
        logits = self.fc(combined)

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return loss, logits
        else:
            return logits



## Extensions of the Bert Architecture

In [ ]:
'''
Standard BERT Architecture
Used as the baseline for our experiments
'''
class BERT_Baseline(nn.Module):
    def __init__(self, bert_model_name, num_labels, class_weights=None):
        super(BERT_Baseline, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained(
            bert_model_name,
            num_labels=num_labels
        )
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask, labels=None):
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            outputs = self.bert(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
            return loss, logits
        else:
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            return outputs.logits     


class BERT_CNN(nn.Module):
    def __init__(self, bert_model_name, num_labels, dropout_rate=0.5, class_weights=None):
        super(BERT_CNN, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout_bert = nn.Dropout(dropout_rate)
        self.dropout_cnn = nn.Dropout(dropout_rate)
        self.conv1d = nn.Conv1d(
            in_channels=self.bert.config.hidden_size, 
            out_channels=128,  
            kernel_size=3,  
            padding=1 
        )
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.classifier = nn.Linear(128, num_labels)
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state 
        sequence_output = self.dropout_bert(sequence_output)
        cnn_output = self.conv1d(sequence_output.permute(0, 2, 1))  
        cnn_output = self.relu(cnn_output)
        cnn_output = self.maxpool(cnn_output)
        pooled_output = torch.mean(cnn_output, dim=2)  
        pooled_output = self.dropout_cnn(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=class_weights) # for balancing the classes
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return loss, logits

        return logits


class BERT_CNN_Multi(nn.Module):
    def __init__(self, bert_model_name, num_labels, dropout_rate=0.5, class_weights=None):
        super(BERT_CNN_Multi, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout_bert = nn.Dropout(dropout_rate)
        self.dropout_cnn = nn.Dropout(dropout_rate)
        self.conv1d = nn.Conv1d(
            in_channels=self.bert.config.hidden_size,  
            out_channels=256,  
            kernel_size=3,  
            padding=1 
        )
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.conv1d_2 = nn.Conv1d(
            in_channels=256,  
            out_channels=256,  
            kernel_size=3,  
            padding=1 
        )
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.conv1d_3 = nn.Conv1d(
            in_channels=256, 
            out_channels=128, 
            kernel_size=3,  
            padding=1 
        )
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.classifier = nn.Linear(128, num_labels)
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  
        sequence_output = self.dropout_bert(sequence_output)
        cnn_output1 = self.conv1d(sequence_output.permute(0, 2, 1))  
        cnn_output1 = self.relu(cnn_output1)
        cnn_output1 = self.maxpool(cnn_output1)
        cnn_output2 = self.conv1d_2(cnn_output1)  
        cnn_output2 = self.relu2(cnn_output2)
        cnn_output2 = self.maxpool2(cnn_output2)
        cnn_output3 = self.conv1d_3(cnn_output2)  
        cnn_output3 = self.relu3(cnn_output3)
        cnn_output3 = self.maxpool3(cnn_output3)
        pooled_output = torch.mean(cnn_output3, dim=2)  
        pooled_output = self.dropout_cnn(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=class_weights) 
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return loss, logits

        return logits


class BERT_FC(nn.Module):
    def __init__(self, bert_model_name, num_labels, hidden_dim=256, dropout_rate=0.5, class_weights=None):
        super(BERT_FC, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, hidden_dim)
        #self.leakyrelu1 = nn.LeakyReLU()
        self.tanh1 = nn.Tanh()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        #self.leakyrelu2 = nn.LeakyReLU()
        self.tanh2 = nn.Tanh()
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        #self.leakyrelu3 = nn.LeakyReLU()
        self.tanh3 = nn.Tanh()
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        #self.leakyrelu4 = nn.LeakyReLU()
        self.tanh4 = nn.Tanh()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc5 = nn.Linear(hidden_dim, num_labels)
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        x = self.fc1(cls_output)
        #x = self.leakyrelu1(x)
        x = self.tanh1(x)
        x = self.fc2(x)
        #x = self.leakyrelu2(x)
        x = self.tanh2(x)
        x = self.fc3(x)
        #x = self.leakyrelu3(x)
        x = self.tanh3(x)
        x = self.fc4(x)
        #x = self.leakyrelu4(x)
        x = self.tanh4(x)
        x = self.dropout(x)
        logits = self.fc5(x)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(logits.view(-1, self.fc5.out_features), labels.view(-1))
            return loss, logits

        return logits


class BERT_LSTM(nn.Module):
    def __init__(self, bert_model_name, num_labels, hidden_dim=256, dropout_rate=0.5, class_weights=None):
        super(BERT_LSTM, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout_bert = nn.Dropout(dropout_rate)
        self.lstm = nn.LSTM(
            input_size=self.bert.config.hidden_size, 
            hidden_size=hidden_dim, 
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        self.attention_layer = nn.Linear(512, 1)
        self.dropout_lstm = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(hidden_dim*2, num_labels)
        self.class_weights = class_weights
    
    def attention_net(self, lstm_output):
        attention_weights = torch.tanh(self.attention_layer(lstm_output))
        attention_weights = torch.softmax(attention_weights, dim=1)
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        return context_vector

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout_bert(sequence_output)
        #x, (h_n, c_n) = self.lstm(sequence_output)
        lstm_output, _ = self.lstm(sequence_output)
        context_vector = self.attention_net(lstm_output)
        pooled_output = self.dropout_lstm(context_vector)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return loss, logits

        return logits


class BERT_RNN(nn.Module):
    def __init__(self, bert_model_name, num_labels=2, hidden_dim=256, dropout_rate=0.1, class_weights=None):
        super(BERT_RNN, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hidden_size = self.bert.config.hidden_size
        self.rnn = nn.RNN(self.hidden_size, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
        self.gelu = nn.GELU()
        self.classifier = nn.Linear(hidden_dim*2, num_labels)
        self.dropout = nn.Dropout(dropout_rate)
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        x, _ = self.rnn(sequence_output)
        x = torch.mean(x, dim=1)
        x = self.gelu(x)
        x = self.dropout(x)
        logits = self.classifier(x)  

        if labels is not None:
          if self.class_weights is not None: 
              self.class_weights = self.class_weights.to(input_ids.device)  
              loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
          loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
          return loss, logits

        return logits

## Metrics

In [ ]:
def compute_metrics(eval_pred, trial=None, trainer=None):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)  

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    f2 = fbeta_score(labels, preds, beta=2, average='binary')
    mcc = matthews_corrcoef(labels, preds)
    mcc_normalized = (mcc + 1) / 2
    S = (f2 + mcc_normalized) / 2

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "f2": f2,
        "mcc": mcc,
        "mcc_normalized": mcc_normalized,
        "S": S,
    }

    if trial is not None and metrics["precision"] < 0.01:
        trial.report(metrics["f1"], step=trainer.state.epoch)
        raise optuna.exceptions.TrialPruned()

    return metrics

## Training with Hyperparameter Search finetuning (OPTUNA)

In [ ]:
# ------- So that the Optuna Trial can be stopped under varying conditions -------
class EarlyStopStudy(Exception):
    pass

In [ ]:
def train_loop(model, train_loader, val_loader, optimizer, device, epoch=None, tokenizer=None):
    model.train()
    total_train_loss = 0

    print(f"\n[Epoch {epoch}] Training...")
    train_progress = tqdm(train_loader, desc="Train", leave=False)

    for batch in train_progress:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        graphs = batch["gnn_graph"].to(device)

        loss, _ = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            graphs=graphs,
            labels=labels
        )

        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_progress.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)

    # ------- Validation -------
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    print(f"[Epoch {epoch}] Validating...")
    val_progress = tqdm(val_loader, desc="Val", leave=False)

    with torch.no_grad():
        for batch in val_progress:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            graphs = batch["gnn_graph"].to(device)

            loss, logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                graphs=graphs,
                labels=labels
            )
            total_val_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(logits.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            val_progress.set_postfix(loss=loss.item())

    avg_val_loss = total_val_loss / len(val_loader)

    eval_pred = (np.array(all_preds), np.array(all_labels))
    metrics = compute_metrics(eval_pred)

    print(f"[Epoch {epoch}] Results:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")
    print(f"  Precision:  {metrics['precision']:.4f}")
    print(f"  Recall:     {metrics['recall']:.4f}")
    print(f"  F1 Score:   {metrics['f1']:.4f}")
    print(f"  F2 Score:   {metrics['f2']:.4f}")
    print(f"  MCC:        {metrics['mcc']:.4f}")
    print(f"  S Score:    {metrics['S']:.4f}")

    return avg_train_loss, avg_val_loss, metrics

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

best_score = 0.0
best_model_path = ""

torch.cuda.empty_cache()

def objective(trial):
    global best_score, best_model_path

    learning_rate = trial.suggest_float("learning_rate", 3e-5, 1e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5) # minimizes overfitting
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4)

    num_training_steps = len(train_dataset) // batch_size * num_train_epochs
    warmup_steps = int(0.08 * num_training_steps)

    # ------- Model -------
    #'''   
    model = BertGNNClassifier(
        bert_model_name=MODEL_NAME,
        hidden_dim=128,
        num_labels=2,
        dropout_rate=dropout_rate,
        class_weights=class_weights
    ).to(device)
    #'''   
    '''
    model = BERT_CNN(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        dropout_rate=dropout_rate,
        class_weights=class_weights
    ).to(device)
    '''
    '''
    model = BERT_CNN_Multi(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        dropout_rate=dropout_rate,
        class_weights=class_weights
    ).to(device)
    '''
    '''
    model = BERT_RNN(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        class_weights=class_weights
    ).to(device)
    '''
    '''
    model = BERT_FC(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        class_weights=class_weights
    ).to(device)
    '''
    '''
    model = BERT_LSTM(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        class_weights=class_weights
    ).to(device)
    '''
    '''
    model = BERT_Baseline(
        bert_model_name=MODEL_NAME,
        num_labels=2,
        class_weights=class_weights
    ).to(device)
    '''
    print(model)

    # Training args
    training_args = TrainingArguments(
        output_dir="/content/optuna_trials",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        logging_dir="/content/logs_optuna",
        eval_strategy="epoch",
        save_strategy="epoch",
        report_to="none", 
        warmup_steps=warmup_steps,
        disable_tqdm=False, # shows Progress Bar
        fp16=True,
        load_best_model_at_end=True,
        metric_for_best_model="S",  
        greater_is_better=True,
    )


    '''
    # ------- Standard Trainer -------
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        #compute_metrics=compute_metrics,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, trial, trainer)
    )

    # Train
    trainer.train()

    # Evaluate
    eval_results = trainer.evaluate()
    score = eval_results["eval_S"]

    # Report the metric to Optuna
    trial.report(eval_results["eval_S"], step=0)

    # Custom pruning condition
    if eval_results["eval_S"] > 0.65:
        print(f"Early stopping study: Trial {trial.number} achieved S = {eval_results['eval_S']:.4f}")

        # Save model if desired
        output_dir = f"C:/Users/fauts/Felix_MA/Projektstudium/results/best_model_trial_{trial.number}"
        os.makedirs(output_dir, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))
        tokenizer.save_pretrained(output_dir)
        best_trial = trial

        raise EarlyStopStudy()

    # Only save if it's the best so far
    if score > best_score:
        best_score = score
        best_model_path = f"C:/Users/fauts/Felix_MA/Projektstudium/results/best_model_trial_{trial.number}"

        output_dir = f"C:/Users/fauts/Felix_MA/Projektstudium/results/best_model_trial_{trial.number}"
        os.makedirs(output_dir, exist_ok=True)

        # Save model weights
        torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))

        # Save tokenizer (this is still a vanilla tokenizer)
        tokenizer.save_pretrained(output_dir)

        print(f"New best model saved to {best_model_path} with score: {score:.4f}")

    return score
    '''

    # ------- Graph Trainer -------
    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    metrics = []

    # Training loop
    for epoch in range(num_train_epochs):
        train_loss, val_loss, metrics = train_loop(
            model, train_loader, val_loader, optimizer, device, epoch=epoch, tokenizer=tokenizer
        )
        #val_loss, val_acc = eval_loop(model, val_loader, device)
        #print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    score = metrics["S"] 

    trial.report(score, step=0)

    trial.report(metrics["S"], step=0)

    # ------- Custom pruning condition -------
    if score > 0.68:
        print(f"Early stopping study: Trial {trial.number} achieved S = {metrics['S']:.4f}")

        output_dir = f"/content/best_model_trial_{trial.number}"
        os.makedirs(output_dir, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))
        tokenizer.save_pretrained(output_dir)
        best_trial = trial

        raise EarlyStopStudy()

    if score > best_score:
        best_score = score
        best_model_path = f"/content/best_model_trial_{trial.number}"

        output_dir = f"/content/best_model_trial_{trial.number}"
        os.makedirs(output_dir, exist_ok=True)

        torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))

        tokenizer.save_pretrained(output_dir)

        print(f"New best model saved to {best_model_path} with score: {score:.4f}")

    return score


study = optuna.create_study(direction="maximize", study_name="bert_gcn")
try:
  study.optimize(objective, n_trials=10)
except EarlyStopStudy:
    print("Study stopped early due to S > 0.68")



print("Best trial:")
print(study.best_trial)
joblib.dump(study, "bert_cnn_optuna_study.pkl")
print("Study saved")

## Training with fixed Parameters

In [ ]:
dropout_rate=0.15017202428568366
training_args = TrainingArguments(
    output_dir="/content/optuna_trials",
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=3.399858326654117e-05,
    weight_decay=0.05784362654134335,
    logging_dir="/content/logs_optuna",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none", 
    disable_tqdm=False,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="S", 
)

torch.cuda.empty_cache()


# ------- Model -------
'''   
model = BertGNNClassifier(
    bert_model_name=MODEL_NAME,
    hidden_dim=128,
    num_labels=2,
    dropout_rate=dropout_rate,
    class_weights=class_weights
).to(device)
'''   
#'''
model = BERT_CNN(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    dropout_rate=dropout_rate,
    class_weights=class_weights
).to(device)
#'''
'''
model = BERT_CNN_Multi(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    dropout_rate=dropout_rate,
    class_weights=class_weights
).to(device)
'''
'''
model = BERT_RNN(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    class_weights=class_weights
).to(device)
'''
'''
model = BERT_FC(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    class_weights=class_weights
).to(device)
'''
'''
model = BERT_LSTM(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    class_weights=class_weights
).to(device)
'''
'''
model = BERT_Baseline(
    bert_model_name=MODEL_NAME,
    num_labels=2,
    class_weights=class_weights
).to(device)
'''
'''
model = BERTDynamicGNNClassifier(
    hidden_dim=768,
    dropout_rate=dropout_rate,
    num_labels=2,
    bert_model_name=MODEL_NAME,
    class_weights=class_weights
).to(device)
'''

print(model)
# ------- Standard Trainer -------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda eval_pred: compute_metrics(eval_pred,trainer)
)

# ------- Train -------
trainer.train()

# ------- Evaluate -------
eval_results = trainer.evaluate()
score = eval_results["eval_S"]

best_model_path = f"path/to/your//best_model_fixed_parameters"

output_dir = f"path/to/your/best_model_fixed_parameters"
os.makedirs(output_dir, exist_ok=True)

# ------- Save model weights -------
torch.save(model.state_dict(), os.path.join(output_dir, "model_state_dict.pt"))

tokenizer.save_pretrained(output_dir)

print(f"New best model saved to {best_model_path} with score: {score:.4f}")

## Evaluation on pandas dataframe (test)

In [ ]:
csv_path = "path/to/your/hatespeech_dataset"
df = pd.read_csv(csv_path)

df = df[["text", "label_hs", "split_all"]]
df = df.rename(columns={"label_hs": "labels"})
df["text"] = df["text"].astype(str)

df["text"] = df["text"].apply(clean_text)

train_df = df[df["split_all"] == "train"][["text", "labels"]].reset_index(drop=True)
test_df = df[df["split_all"] == "test"][["text", "labels"]].reset_index(drop=True)
val_df = df[df["split_all"] == "val"][["text", "labels"]].reset_index(drop=True)

model_path = "path/to/your/best_model_fixed_parameters"
tokenizer = AutoTokenizer.from_pretrained(model_path)
#print(f"Tokenizer vocab_size: {len(tokenizer)}")
print(f"Tokenizer vocab_size: {tokenizer.vocab_size}")

model = BERT_CNN(bert_model_name=MODEL_NAME, num_labels=2)

model.load_state_dict(torch.load(f"{model_path}/model_state_dict.pt", map_location=torch.device("cuda")))
model.to(device)
model.eval()  
print("Model and tokenizer loaded successfully!")

predictions = []
true_labels = []
if "test_df" not in locals():
    raise ValueError("test_df is not defined. Make sure to load your test dataset.")

for text, label in zip(test_df["text"], test_df["labels"]):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128) 
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs)  

    probs = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()

    predictions.append(predicted_class)
    true_labels.append(label)

results_df = pd.DataFrame({"Text": test_df["text"], "True Label": true_labels, "Predicted Label": predictions})
print(results_df.head())
#results_df.to_csv("projektstudium/results", index=False) # Pfad anpassen
print("Predictions gespeichert unter: /content/results") # Pfad anpassen


def compute_metrics_from_df(results_df):
    labels = results_df["True Label"].values
    preds = results_df["Predicted Label"].values

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    f2 = fbeta_score(labels, preds, beta=2, average='binary')
    mcc = matthews_corrcoef(labels, preds)
    mcc_normalized = (mcc + 1) / 2
    S = (f2 + mcc_normalized) / 2 

    cm = confusion_matrix(labels, preds)
    class_labels = ["No Hate-Speech", "Hate-Speech"]

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "f2": f2,
        "mcc": mcc,
        "mcc_normalized": mcc_normalized,
        "S": S,
    }

    return metrics

metrics = compute_metrics_from_df(results_df)
print("Evaluation Metrics:", metrics)